# Numerical tests for paper

The goal of this notebook is to create a train of ideas and experiment to reflect the logic expressed in the numerical experiments section of the paper to indicate that:
- We can use coherence on an array of sensors to detect events (impulsive and emergent)
- SVD and QR based coherence analyses are comparable in performance
- The relationship between SVD and QR based coherence analyses depends on source configuration and noise level
- QR based coherence analyses are more computationally efficient than SVD based analyses

## Numerical Experiments

In comparing the QR and SVD based coherence analyses, we test detectability depend on the following factors:
- Noise amplitude and covariance (white, colored, spatially correlated)
- Source type (impulsive, emergent)
- Number of events
- Times of arrival, separation in frequency content, and source locations for multiple events

We first test the noise with a single impulsive source, and then a single emergent source. We then combine the two source types and test multiple events coming from different frequency contents. The intuition is that once the frequency contents are separated, other factors apart from noise level should not affect the performance of the coherence analyses. Next we test the same events occurring at different times to see if time separation affects the performance. Finally, we test multiple events of similar frequency content occurring at the same and different times.



In [ ]:
import os
import sys
import time
from typing import Optional

import deepwave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import hilbert
import seaborn as sns
import torch
from torch import Tensor
from deepwave import scalar

sys.path.append("../")
sys.path.append(os.path.join(os.path.dirname(""), os.pardir, os.pardir))
import coherence_analysis.utils as f

In [ ]:
tick_size = 14
label_size = 16
title_size = 18
legend_size = 12
colors = ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]
okabe_ito = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # bluish green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # vermillion
    "#CC79A7",  # reddish purple
]
sns.set_theme(style="whitegrid", context="paper")
dpi = 600

### Source Wavelets

`Deepwave` only has a Ricker wavelet source implementation. We implement other wavelet types here for more variety in source wavelets. We implement a minimum-phase wavelet version of the *Ormsby wavelet* for impulsive sources and a *Berlage wavelet* for emergent sources.

In [ ]:
fc = 3
alpha = 5
dt = 0.005
t = np.arange(-0.2, 5, dt)

# w = berlage_wavelet(t, fc, alpha, n=3)
w = f.berlage_wavelet(fc, len(t), dt, 0.2, alpha=alpha, n=10, phi=0)
w = w.numpy()
# w /= np.max(np.abs(w))*150  # normalize

plt.figure(figsize=(12, 5), dpi=dpi)
plt.subplot(121)
plt.plot(t, w)
plt.title("Berlage Wavelet", fontsize=title_size)
plt.xlabel("Time (s)", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Berlage Wavelet")
plt.xlabel("Frequency (Hz)", fontsize=label_size)
plt.ylabel("Magnitude", fontsize=label_size)
plt.title("Magnitude of the Berlage wavelet spectrum", fontsize=title_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);
# plt.legend(fontsize=label_size)

In [ ]:
# corner frequencies
corner_freqs = 10, 20, 40, 50  # Hz
dt = 0.005
nt = 2 / dt
t = np.linspace(-2, 2, int(nt))

w = f.min_phase_ormsby_wavelet(corner_freqs, int(nt), dt, peak_time=1)
w = w.numpy()

plt.figure(figsize=(12, 5), dpi=dpi)
plt.subplot(121)
plt.plot(t, w)
plt.title("Ormsby Wavelet", fontsize=title_size)
plt.xlabel("Time (s)", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
# plt.grid(True)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Ormsby Wavelet")
plt.xlabel("Frequency (Hz)", fontsize=label_size)
plt.ylabel("Magnitude", fontsize=label_size)
plt.title("Magnitude of the Ormsby wavelet spectrum", fontsize=title_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);
# plt.legend()

In [ ]:
# parameters
fc = 20  # Hz
alpha = 200
dt = 0.005
t = np.arange(-1, 1, dt)

w = f.gabor_wavelet(t, fc, alpha)
w = f.minimum_phase_wavelet(w)

plt.figure(figsize=(12, 5), dpi=dpi)
plt.subplot(121)
plt.plot(t, w)
plt.title("Gabor Wavelet", fontsize=title_size)
plt.xlabel("Time (s)", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Gabor Wavelet")
plt.xlabel("Frequency (Hz)", fontsize=label_size)
plt.ylabel("Magnitude", fontsize=label_size)
plt.title("Magnitude of the Gabor wavelet spectrum", fontsize=title_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);
# plt.legend()

Choose which device we wish to run on, specify the size of the model, and load it:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Set-up the general field parameters

Specify dimensions of model and load velocity model. Here, we are using the marmousi model because it has a complex velocity structure that will generate complicated wavefields.

In [ ]:
ny = 2301
nx = 751
dx = 0.5

marmousi_offset_left = 200
marmousi_offset_right = 0
ny = min(2301 - marmousi_offset_left - marmousi_offset_right, ny)

assert marmousi_offset_left + marmousi_offset_right + ny <= 2301, (
    "Offsets exceed the model size."
)
assert min(marmousi_offset_left, marmousi_offset_right) == 0, (
    "To stay true to offsets, one side must be unrestricted i.e set to 0."
)

v = torch.from_file("marmousi_vp.bin", size=(ny + marmousi_offset_left) * nx)
v = v[marmousi_offset_left * nx :].reshape(ny, nx).to(device)

Make a plot of the velocity model:

In [ ]:
plt.imshow(v.cpu().numpy().T, cmap="seismic", aspect="auto")
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.colorbar()

Specify receiver locations. Here, we are using a linear array of receivers at a shallow depth to simulate a surface seismic survey. `nshots` is set to 1 to simulate a single shot gather.

In [ ]:
n_shots = 1

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

### Set up source parameters and run simulation

Specify source locations and source wavelet. Here, we are using two impulsive sources to simulate seismic events. We do this step multiple times to simulate different source configurations.

#### Configuration 1: Single impulsive source
The first configuration uses a single source. We first use a minimum-phase Ormsby wavelet as the impulsive source wavelet.

In [ ]:
n_sources_per_shot = 1
d_source = 1500  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

corner_freqs = [5, 15, 35, 45]  # Hz
# corner_freqs = [5, 15, 20, 30]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

# source_amplitudes
source_amplitudes = (
    f.min_phase_ormsby_wavelet(corner_freqs, int(nt), dt, peak_time=peak_time)
    .repeat(n_shots, n_sources_per_shot, 1)
    .to(device)
)

In [ ]:
plt.figure(figsize=(10, 6), dpi=dpi)
plt.imshow(v.cpu().numpy().T[:600], cmap="cividis", aspect="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=legend_size)
cbar.set_label("Velocity (m/s)", size=label_size, weight="bold")
# Receiver locations visualization
plt.scatter(
    receiver_locations[0, 25:-25, 0].cpu().numpy(),
    receiver_locations[0, 25:-25, 1].cpu().numpy(),
    marker="v",
    c="yellow",
    s=30,
    label="Receivers",
)
# Source locations visualization
plt.scatter(
    source_locations[0, :, 0].cpu().numpy(),
    source_locations[0, :, 1].cpu().numpy(),
    c="red",
    s=30,
    label="Sources",
)
# plt.scatter(200, 500, c="red", s=30)  # Additional source location
plt.title(
    "Velocity Model with Receiver and Source Locations", fontsize=label_size
)
plt.xlabel("X (m)", fontsize=label_size)
plt.ylabel("Z (m)", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(fontsize=legend_size)

### Propagate wavefield to generate synthetic receiver data

This function returns:
- wavefield_nt: The wavefield at the final time step.
- wavefield_ntm1: The wavefield at the second-to-last time step.
- psiy_ntm1: PML-related wavefield.
- psix_ntm1: PML-related wavefield.
- zetay_ntm1: PML-related wavefield.
- zetax_ntm1: PML-related wavefield.
- receiver_amplitudes: The receiver amplitudes. Empty if no receivers were specified.

In [ ]:
save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "simulated_data"
)
file_path = save_directory + "/" + "config1_2hz_1_source.pt"  # ricker
file_path = save_directory + "/" + "config1_5_15_35_45_hz_1_source_ormsby.pt"
# file_path = save_directory + "/" + "config1_5_15_20_30_hz_1_source_ormsby.pt"
file_path = (
    save_directory + "/" + "config1_5_15_35_45_hz_1_source_ormsby1.pt"
)  # LOC 200, 500
if os.path.exists(file_path):
    receiver_amplitudes = torch.load(file_path)
else:
    out = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes = out[-1]
    torch.save(receiver_amplitudes, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes[0], torch.tensor([0.1, 0.995]).to(device)
)
plt.figure(figsize=(14, 6), dpi=dpi)
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.xlabel("Receiver number", fontsize=label_size)
plt.ylabel("Time sample", fontsize=label_size)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=legend_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes.cpu()[0, 19])
plt.title("Receiver 20", fontsize=title_size)
plt.xlabel("Time", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);

In [ ]:
plt.figure(figsize=(18, 6), dpi=dpi)
coherence_data = receiver_amplitudes[0].cpu().numpy()

cov_len = 10
noise = f.get_noise(coherence_data, cov_len)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data)) * 0.1
coherence_data = coherence_data + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes[0], torch.tensor([0.1, 0.995]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.xlabel("Receiver number", fontsize=label_size)
plt.ylabel("Time sample", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.xlabel("Receiver number", fontsize=label_size)
# plt.ylabel("Time sample", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.colorbar()
plt.subplot(1, 3, 3)
plt.plot(coherence_data[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20", fontsize=title_size)
plt.xlabel("Time", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylabel("Amplitude", fontsize=label_size)
plt.legend(fontsize=legend_size)

In [ ]:
w = coherence_data[19]

plt.figure(figsize=(12, 5), dpi=dpi)
plt.subplot(121)
plt.plot(w)
plt.title("Receiver 20", fontsize=title_size)
plt.xlabel("Time (s)", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
# spectra /= np.max(np.abs(spectra))
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o")
plt.xlabel("Frequency (Hz)", fontsize=label_size)
plt.ylabel("Magnitude", fontsize=label_size)
plt.title("Magnitude of the receiver 20 spectrum", fontsize=title_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);
# plt.legend()

In [ ]:
w = noise[19]

plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(w)
plt.title("Ormsby Wavelet")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid(True)

plt.subplot(122)
spectra = np.fft.rfft(w)
# spectra /= np.max(np.abs(spectra))
frequencies = np.fft.rfftfreq(len(w), dt)
plt.plot(frequencies, np.absolute(spectra), "-o", label="Ormsby Wavelet")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Magnitude of the Ormsby wavelet spectrum")
plt.legend()

In [ ]:
xx = 1
win_len = 1 / xx
overlap = 0
samples_per_sec = 1 / dt
nr, nc = coherence_data.shape
coherence_data = coherence_data[:, : nc // xx]
noise = noise[:, : nc // xx]

t0 = time.time()
event_detection_qr, qrs, frequencies = f.coherence(
    coherence_data, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd, svds, frequencies = f.coherence(
    coherence_data, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
y_limit = 0.9
# num_frames = coherence_2.shape[0]
# f_plot = np.linspace(0, 124, num_frames)
f_plot = frequencies
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd[1:last_freq_index],
    "-o",
    label="Standard",
    color="darkviolet",
)
plt.ylim(0, y_limit)
plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    "-o",
    label="Standard",
    color="darkviolet",
)
plt.ylim(0, y_limit)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
norm_win_spectra, _ = f.normalised_windowed_spectra(
    coherence_data, win_len, overlap, sample_interval=1 / samples_per_sec
)
norm_win_spectra_noise, _ = f.normalised_windowed_spectra(
    noise,
    win_len,
    overlap,
    sample_interval=1 / samples_per_sec,
)

i = 19
Q, R = np.linalg.qr(norm_win_spectra[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Comparison with STA/LTA method

In [ ]:
stalta_data = f.stalta_freq(
    coherence_data,
    500,
    75,
)
xax = np.array(range(stalta_data.shape[1])) * dt
plt.plot(xax, np.mean(stalta_data, axis=0), color=colors[0], linewidth=1.5)

### Configuration 1: Part II - Emergent source

Next, we simulate an emergent source using a Berlage wavelet.

In [ ]:
n_sources_per_shot = 1
d_source = 1500  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m

freq = 3
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

source_amplitudes = (
    f.berlage_wavelet(freq, nt, dt, peak_time, alpha=5, n=10, phi=0)
    .repeat(n_shots, n_sources_per_shot, 1)
    .to(device)
)

In [ ]:
save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "simulated_data"
)
file_path = save_directory + "/" + "config1_10hz_1_source.pt"
file_path = save_directory + "/" + "config1_3hz_emergent_source_berlage.pt"
file_path = (
    save_directory + "/" + "config1_3hz_emergent_source_berlage1.pt"
)  # LOC 200, 500
if os.path.exists(file_path):
    receiver_amplitudes_2 = torch.load(file_path)
else:
    out_2 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=freq,
    )
    receiver_amplitudes_2 = out_2[-1]
    torch.save(receiver_amplitudes_2, file_path)

In [ ]:
plt.figure(figsize=(17, 6), dpi=dpi)
coherence_data_2 = receiver_amplitudes_2[0].cpu().numpy()

cov_len = 1
sig_to_noise = 1
noise = f.get_noise(coherence_data_2, cov_len)
noise = noise / np.max(np.abs(noise))
noise = noise * np.max(np.abs(coherence_data_2)) * sig_to_noise
coherence_data_2 = coherence_data_2 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_2[0], torch.tensor([0.1, 0.995]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_2.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_2[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20", fontsize=title_size)
plt.xlabel("Time", fontsize=label_size)
plt.ylabel("Amplitude", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.legend(fontsize=legend_size)

In [ ]:
xx = 1
win_len = 1 / xx
overlap = 0
samples_per_sec = 1 / dt
nr, nc = coherence_data_2.shape
coherence_data_2 = coherence_data_2[:, : nc // xx]
noise = noise[:, : nc // xx]

t0 = time.time()
event_detection_qr_2, qrs_2, frequencies = f.coherence(
    coherence_data_2, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_2, svds_2, frequencies = f.coherence(
    coherence_data_2, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
# f_plot = np.linspace(0, 124, num_frames)
f_plot = frequencies
plt.figure(figsize=(14, 6), dpi=dpi)
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_2[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_2[1:last_freq_index],
    "-o",
    label="Standard",
    color="darkviolet",
)
# plt.ylim(0, 0.35)

plt.ylabel("Detection parameter", fontsize=label_size)
plt.xlabel("Frequency", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("Event", fontsize=title_size)
plt.legend(fontsize=legend_size)
# plt.xlim(0, 5)
plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    "-o",
    label="Standard",
    color="darkviolet",
)

plt.ylim(0, 0.35)
plt.ylabel("Detection parameter", fontsize=label_size)
plt.xlabel("Frequency", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.title("Noise", fontsize=title_size)
plt.legend(fontsize=legend_size)

In [ ]:
# norm_win_spectra, _ = f.normalised_windowed_spectra(
#     coherence_data_2, win_len, overlap, sample_interval=1 / samples_per_sec
# )
norm_win_spectra_noise, _ = f.normalised_windowed_spectra(
    noise,
    win_len,
    overlap,
    sample_interval=1 / samples_per_sec,
)

i = 3
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.figure(figsize=(10, 5), dpi=dpi)
plt.subplot(1, 2, 1)
plt.plot(qrs_2[i] / np.sum(qrs_2[i]), "-o", color=colors[4], label="signal")
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=label_size)
plt.xlabel("Snapshot in time", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([0, 0.85])
plt.legend(fontsize=legend_size)
plt.subplot(1, 2, 2)
plt.plot(svds_2[i] / np.sum(svds_2[i]), "-o", color=colors[4], label="signal")
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=label_size)
plt.xlabel("Snapshot in time", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([0, 0.85])
plt.legend(fontsize=legend_size)

In [ ]:
sig_to_noise = 5
corner_freqs = [5, 15, 35, 45]  # Hz

save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "results"
)
file_path = (
    save_directory
    + "/"
    + "impulsive_decay_df_sigtonoise"
    + str(sig_to_noise)
    + "_freqs_5_15_35_45.pkl"
)

if os.path.exists(file_path):
    imp_decay_df = pd.read_pickle(file_path)
else:
    imp_decay_df = f.coherence_decay_test(
        receiver_amplitudes[0].cpu().numpy()[25:-25],
        win_len=win_len,
        overlap=overlap,
        sample_interval=dt,
        signal_to_noise=sig_to_noise,
        cov_len=cov_len,
        event_freq_range=(corner_freqs[1], corner_freqs[2]),
        num_of_sims=200,
    )
    imp_decay_df.to_pickle(file_path)

imp_decay_df_normalized = imp_decay_df.copy()
imp_decay_df_normalized[0] = imp_decay_df_normalized[0] / 300

In [ ]:
sig_to_noise = 0.25
freq = 3
cov_len = 1

save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "results"
)
file_path = (
    save_directory
    + "/"
    + "emergent_decay_df_sigtonoise"
    + str(sig_to_noise)
    + "_freq_"
    + str(freq)
    + ".pkl"
)

if os.path.exists(file_path):
    em_decay_df = pd.read_pickle(file_path)
else:
    em_decay_df = f.coherence_decay_test(
        receiver_amplitudes_2[0].cpu().numpy()[25:-25],
        win_len=win_len,
        overlap=overlap,
        sample_interval=dt,
        signal_to_noise=sig_to_noise,
        cov_len=cov_len,
        event_freq_range=freq,
        num_of_sims=200,
    )
    em_decay_df.to_pickle(file_path)
em_decay_df_normalized = em_decay_df.copy()
em_decay_df_normalized[0] = em_decay_df_normalized[0] / 300

In [ ]:
plt.figure(figsize=(8, 6), dpi=dpi)

sns.lineplot(
    data=imp_decay_df_normalized,
    x="Index",
    y=0,
    hue="Method",
    style="Data_Label",
    linewidth=2.5,
    markers=True,
    markersize=8,
    dashes=True,  # cleaner, optional
    err_style="bars",
    errorbar=("sd", 2),
    alpha=0.9,
    palette="colorblind",  # okabe_ito,  # "deep" stronger colors than "pastel"
)

plt.xlabel("Index", fontsize=label_size)
plt.ylabel("Detection Parameter", fontsize=label_size)
plt.title("Impulse Decay Comparison", fontsize=title_size)

plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

plt.legend(
    # title="Method / Data Label",
    frameon=True,
    # bbox_to_anchor=(1.05, 1),
    # loc="upper left",
    fontsize=12,
)

plt.tight_layout()
# plt.show()

In [ ]:
em_decay_df_normalized["Signal_Type"] = "Persistent"
imp_decay_df_normalized["Signal_Type"] = "Impulsive"
combined_decay_df = pd.concat(
    [imp_decay_df_normalized, em_decay_df_normalized]
)

combined_decay_df.loc[combined_decay_df["Method"] == "qr", "Method"] = (
    "QR Approximation"
)
combined_decay_df.loc[combined_decay_df["Method"] == "svd", "Method"] = (
    "Standard"
)
combined_decay_df.loc[
    combined_decay_df["Data_Label"] == "signal", "Data_Label"
] = "Signal"
combined_decay_df.loc[
    combined_decay_df["Data_Label"] == "noise", "Data_Label"
] = "Noise"

In [ ]:
plt.figure(dpi=dpi)
g = sns.FacetGrid(
    combined_decay_df,
    col="Method",
    # hue="Method",
    # style="Data_Label",
    height=4.5,
    aspect=1.2,
    palette="colorblind",
)
g.map_dataframe(
    sns.lineplot,
    x="Index",
    y=0,
    hue="Signal_Type",
    style="Data_Label",
    linewidth=2.5,
    markers=True,
    markersize=8,
    dashes=True,  # cleaner, optional
    err_style="bars",
    errorbar=("sd", 2.5),
    alpha=0.9,
    palette="colorblind",  # okabe_ito,  #"deep" stronger colors than "pastel"
    # legend=False,
    # title=False,
)

# Axis labels
g.set_axis_labels("Index", "Normalized Eigenvalue", fontsize=label_size)

# Facet titles
# g.set_titles(col_template="{col_name}", size=title_size)
g.set_titles("")

# Tick label sizes
for ax in g.axes.flat:
    ax.tick_params(axis="both", labelsize=tick_size)

# Legend fontsize
# g.add_legend(title="Data Label")
g.add_legend(
    fontsize=legend_size, frameon=True, bbox_to_anchor=(0.8, 0.7)
)  # , loc="center left")
# plt.setp(g._legend.get_texts(), fontsize=legend_size)
# plt.setp(g._legend.get_title(), fontsize=13)

plt.text(
    -0.005,
    0.9,
    "(a)",
    fontsize=label_size,
    transform=plt.gcf().transFigure,
    bbox=dict(facecolor="white", alpha=0.9, boxstyle="round,pad=0.1"),
)
plt.text(
    0.452,
    0.9,
    "(b)",
    fontsize=label_size,
    transform=plt.gcf().transFigure,
    bbox=dict(facecolor="white", alpha=0.9, boxstyle="round,pad=0.1"),
)
# plt.tight_layout()

In [ ]:
plt.figure(figsize=(10, 6), dpi=dpi)
sns.lineplot(
    data=em_decay_df_normalized,
    x="Index",
    y=0,
    hue="Method",
    style="Data_Label",
    err_style="bars",
    errorbar=("sd", 3),
    markers=True,
    markersize=8,
    linewidth=2.5,
    palette=okabe_ito,  # "deep" stronger colors than "pastel"
)

plt.xlabel("Index", fontsize=label_size)
plt.ylabel("Value", fontsize=label_size)
plt.title("Emergent Decay Comparison", fontsize=title_size)

plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

plt.legend(
    # title="Method / Data Label",
    frameon=True,
    # bbox_to_anchor=(1.05, 1),
    # loc="upper left",
    fontsize=12,
)

plt.tight_layout()
# plt.show()

In [ ]:
coherence_data = receiver_amplitudes[0].cpu().numpy()
nr, nc = coherence_data.shape

signal_to_noise_list = [2, 1.75, 1.5, 1.25, 1, 0.75, 0.5, 0.25]
cov_len_list = [10, 50, 100, 200]
# cov_len_list = [10, 20, 50, 100]

save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "results"
)
file_path = (
    save_directory + "/" + "noise_test_df_impulsive_freqs_5_15_35_45.pkl"
)

if os.path.exists(file_path):
    df = pd.read_pickle(file_path)
else:
    df = f.noise_test(
        receiver_amplitudes[0].cpu().numpy(),
        win_len=win_len,
        overlap=overlap,
        sample_interval=dt,
        signal_to_noise_list=signal_to_noise_list,
        cov_len_list=cov_len_list,
        event_freq_range=(corner_freqs[1], corner_freqs[2]),
        num_of_sims=200,
    )
    df.to_pickle(file_path)

In [ ]:
coherence_data_2 = receiver_amplitudes_2[0].cpu().numpy()
nr, nc = coherence_data_2.shape

signal_to_noise_list = [2, 1.75, 1.5, 1.25, 1, 0.75, 0.5, 0.25]
cov_len_list = [30, 75, 150, 300]
# cov_len_list = [10, 50, 100, 200]
# cov_len_list = [10, 20, 50, 100]

save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data", "results"
)
file_path = save_directory + "/" + "noise_test_df_emergent_freq_3.pkl"

if os.path.exists(file_path):
    df_2 = pd.read_pickle(file_path)
else:
    df_2 = f.noise_test(
        receiver_amplitudes_2[0].cpu().numpy(),
        win_len=win_len,
        overlap=overlap,
        sample_interval=dt,
        signal_to_noise_list=signal_to_noise_list,
        cov_len_list=cov_len_list,
        event_freq_range=freq,
        num_of_sims=200,
    )
    df_2.to_pickle(file_path)

In [ ]:
g = sns.FacetGrid(
    df[df["Method"] != "ratio"],
    row="Method",
    col="Covariance_Length",
    hue="Data_Label",
    height=4.5,
    aspect=1.2,
    palette="pastel",
)
g.map(
    sns.violinplot,
    "Signal_to_Noise",
    "Detection_Parameter",
    split=True,
    fill=False,
    inner="quart",
)
# place legend
g.add_legend(fontsize=legend_size)

In [ ]:
plt.figure(dpi=dpi)
g = sns.FacetGrid(
    df_2[df_2["Method"] != "ratio"],
    row="Method",
    col="Covariance_Length",
    hue="Data_Label",
    height=4.5,
    aspect=1.2,
    palette="pastel",
)
g.map_dataframe(
    sns.violinplot,
    x="Signal_to_Noise",
    y="Detection_Parameter",
    # hue="Method",
    split=True,
    fill=False,
    inner="quart",
)
# place legend

# g.add_legend(fontsize=10, loc="center")

# Axis labels
g.set_axis_labels("Index", "Value", fontsize=label_size)

# Facet titles
g.set_titles(col_template="{col_name}", size=title_size)

# Tick label sizes
for ax in g.axes.flat:
    ax.tick_params(axis="both", labelsize=tick_size)

# Legend fontsize
# g.add_legend(title="Data Label")
g.add_legend(fontsize=legend_size)
# plt.setp(g._legend.get_texts(), fontsize=legend_size)
# plt.setp(g._legend.get_title(), fontsize=13)

# plt.tight_layout()

## Configuration 2: Multiple sources with different frequency contents

Here, we combine the two source types used previously to simulate multiple events with different frequency contents. We use one impulsive source with a minimum-phase Ormsby wavelet and one emergent source with a Berlage wavelet.

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 0  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

freq = 3
corner_freqs = [10, 20, 40, 50]  # Hz
corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 4 / freq

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

# source_locations[..., 0, 1] = 500
# source_locations[..., 1, 1] = 500

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 1, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
source_amplitudes = (
    torch.cat(
        (
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 1
            ).repeat(n_shots, 1, 1),
            f.berlage_wavelet(
                freq, nt, dt, peak_time, alpha=5, n=10, phi=0
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = save_directory + "/" + "config3_2hz_2_sources.pt"
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent.pt"
)  # base case, 5,15,35,45hz
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent2.pt"
)  # adjust peak time +1s, 5,15,35,45hz
# file_path = (
#     save_directory + "/" + "config2_1_impulsive_1_emergent3.pt"
# )  # adjust peak time +1s, 10,20,40,50hz, normalized wavelets
# file_path = (
#     save_directory + "/" + "config2_1_impulsive_1_emergent4.pt"
# )  # adjust peak time +1s, 10,20,40,50hz, unnormalized wavelets
# file_path = (
#     save_directory + "/" + "config2_1_impulsive_1_emergent5.pt"
# )  # adjust peak time +1s, 10,20,40,50hz, normalized wavelets in frequency domain
file_path = (
    save_directory + "/" + "config2_1_impulsive_1_emergent6.pt"
)  # adjust peak time +1s, 5,15,35,45hz, normalized wavelets in frequency domain, LOC 200, 500
if os.path.exists(file_path):
    receiver_amplitudes_3 = torch.load(file_path)
else:
    out_3 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_3 = out_3[-1]
    torch.save(receiver_amplitudes_3, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_3[0], torch.tensor([0.1, 0.99]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_3.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_3.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_3 = receiver_amplitudes_3[0].cpu().numpy()

cov_len = 1
sig_to_noise = 0.05
noise = f.get_noise(coherence_data_3, cov_len)
noise = noise / np.max(np.abs(noise))

noise = noise * np.max(np.abs(coherence_data_3)) * sig_to_noise
coherence_data_3 = coherence_data_3 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_3[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_3.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_3[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(fontsize=fsize)

In [ ]:
xx = 1
win_len = 1 / xx
xx = 1
coherence_data_3 = coherence_data_3[:, : nc // xx]
noise = noise[:, : nc // xx]
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection_qr_3, _, frequencies = f.coherence(
    coherence_data_3, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_3, svds_3, frequencies = f.coherence(
    coherence_data_3, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
# f_plot = np.linspace(0, 124, num_frames)
f_plot = frequencies
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_3[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_3[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="Standard",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="Standard",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
norm_win_spectra_3, _ = f.normalised_windowed_spectra(
    coherence_data_3, win_len, overlap, sample_interval=1 / samples_per_sec
)
norm_win_spectra_noise, _ = f.normalised_windowed_spectra(
    noise,
    win_len,
    overlap,
    sample_interval=1 / samples_per_sec,
)


i = 3

plt.figure(figsize=(15, 5), dpi=dpi)
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_3[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_3[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
plt.plot(svds_3[i] / np.sum(svds_3[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
# plt.legend(fontsize=tick_size)

## Configuration 3: Same source type, different arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 0  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

# source_locations[..., 0, 1] = 500
# source_locations[..., 1, 1] = 500

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 1, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

source_amplitudes = (
    torch.cat(
        (
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 10
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = (
    save_directory + "/" + "config3_2_impulsive_sources.pt"
)  # 2s spacing
file_path = (
    save_directory + "/" + "config3_2_impulsive_sources1.pt"
)  # 4s spacing
file_path = (
    save_directory + "/" + "config3_2_impulsive_sources2.pt"
)  # 10s spacing
file_path = (
    save_directory + "/" + "config3_2_impulsive_sources3.pt"
)  # 10s spacing LOC 200, 500
if os.path.exists(file_path):
    receiver_amplitudes_4 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.97]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()

cov_len = 1
sig_to_noise = 1
noise = f.get_noise(coherence_data_4, cov_len)
noise = noise / np.max(np.abs(noise))

noise = noise * np.max(np.abs(coherence_data_4)) * sig_to_noise
coherence_data_4 = coherence_data_4 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(fontsize=fsize)

In [ ]:
# plt.figure(figsize=(15, 5))
spectra = np.fft.rfft(receiver_amplitudes_4.cpu()[0, 19])
frequencies = np.fft.rfftfreq(len(receiver_amplitudes_4.cpu()[0, 19]), dt)
plt.plot(frequencies, np.absolute(spectra), "--", label="Gabor Wavelet")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
# plt.title("Magnitude of the Gabor wavelet spectrum")
# plt.legend()

In [ ]:
xx = 1
win_len = 1 / xx
xx = 1
coherence_data_4 = coherence_data_4[:, : nc // xx]
noise = noise[:, : nc // xx]
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection_qr_4, qrs_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_4, svds_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
win_len = 1

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
fsize = 12
last_freq_index = -1
# f_plot = np.linspace(0, 124, num_frames)
f_plot = frequencies
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="SVD approximation",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="SVD approximation",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 21

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
plt.plot(qrs_4[i] / np.sum(qrs_4[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
# plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
plt.plot(svds_4[i] / np.sum(svds_4[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);

### Three impulsive sources with different arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 3
d_source = 0  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

# source_locations[..., 0, 1] = 500
# source_locations[..., 1, 1] = 500
# source_locations[..., 2, 1] = 500

source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 1, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 2, 0] = torch.arange(n_shots) * d_source + first_source

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

source_amplitudes = (
    torch.cat(
        (
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 10
            ).repeat(n_shots, 1, 1),
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 20
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = (
    save_directory + "/" + "config3_3_impulsive_sources.pt"
)  # 10s spacing
file_path = (
    save_directory + "/" + "config3_3_impulsive_sources1.pt"
)  # 10s spacing LOC 200, 500
if os.path.exists(file_path):
    receiver_amplitudes_4 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.97]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()

cov_len = 1
sig_to_noise = 0.75
noise = f.get_noise(coherence_data_4, cov_len)
noise = noise / np.max(np.abs(noise))

noise = noise * np.max(np.abs(coherence_data_4)) * sig_to_noise
coherence_data_4 = coherence_data_4 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(fontsize=fsize)

In [ ]:
xx = 1
win_len = 1 / xx
xx = 1
coherence_data_4 = coherence_data_4[:, : nc // xx]
noise = noise[:, : nc // xx]
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection_qr_4, qrs_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_4, svds_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = frequencies
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="SVD approximation",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    "-o",
    color="darkviolet",
    label="SVD approximation",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 21

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
plt.plot(qrs_4[i] / np.sum(qrs_4[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
# plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
plt.plot(svds_4[i] / np.sum(svds_4[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);

## Configuration 4: Different sources, same arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 1500  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 0, 1] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 1, 0] = (
    torch.arange(n_shots) * d_source + first_source + d_source
)

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
source_amplitudes = (
    torch.cat(
        (
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = save_directory + "/" + "config4_2_impulsive_sources_diff_loc.pt"
file_path = (
    save_directory + "/" + "config4_2_impulsive_sources_diff_loc_350.pt"
)
if os.path.exists(file_path):
    receiver_amplitudes_4 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 19])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()

cov_len = 1
sig_to_noise = 0.1
noise = f.get_noise(coherence_data_4, cov_len)
noise = noise / np.max(np.abs(noise))

noise = noise * np.max(np.abs(coherence_data_4)) * sig_to_noise
coherence_data_4 = coherence_data_4 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
xx = 1
win_len = 1 / xx
xx = 1
coherence_data_4 = coherence_data_4[:, : nc // xx]
noise = noise[:, : nc // xx]
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection_qr_4, qrs_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_4, svds_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = frequencies

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    label="SVD approximation",
    color="darkviolet",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
    color="darkviolet",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 10

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

## Configuration 5: Different sources, different arrival times

In [ ]:
n_shots = 1

n_sources_per_shot = 2
d_source = 1700  # 20 * 4m = 80m
first_source = 200  # 10 * 4m = 40m
source_depth = 500  # 2 * 4m = 8m
# first_source = int(ny/2)
# source_depth = int(2*nx/3)

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = min(n_receivers_per_shot, int(ny / d_receiver))

corner_freqs = [5, 15, 35, 45]  # Hz
nt = 750 * 10
dt = 0.004
peak_time = 2

# source_locations
source_locations = torch.zeros(
    n_shots, n_sources_per_shot, 2, dtype=torch.long, device=device
)

source_locations[..., 0, 1] = 500
source_locations[..., 1, 1] = 500

# source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source
source_locations[:, 1, 0] = (
    torch.arange(n_shots) * d_source + first_source + d_source
)

# receiver_locations
receiver_locations = torch.zeros(
    n_shots, n_receivers_per_shot, 2, dtype=torch.long, device=device
)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    torch.arange(n_receivers_per_shot) * d_receiver + first_receiver
).repeat(n_shots, 1)

# source_amplitudes
source_amplitudes = (
    torch.cat(
        (
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time
            ).repeat(n_shots, 1, 1),
            f.min_phase_ormsby_wavelet(
                corner_freqs, int(nt), dt, peak_time=peak_time + 10
            ).repeat(n_shots, 1, 1),
        ),
        dim=1,
    )
).to(device)

In [ ]:
file_path = (
    save_directory + "/" + "config5_2_impulsive_sources_diff_loc_diff_time.pt"
)  # 2s spacing
file_path = (
    save_directory + "/" + "config5_2_impulsive_sources_diff_loc_diff_time1.pt"
)  # 4s spacing
file_path = (
    save_directory + "/" + "config5_2_impulsive_sources_diff_loc_diff_time2.pt"
)  # 10s spacing
file_path = (
    save_directory + "/" + "config5_2_impulsive_sources_diff_loc_diff_time3.pt"
)  # 10s spacing, event locs 200, 500 and 1900, 500
if os.path.exists(file_path):
    receiver_amplitudes_4 = torch.load(file_path)
else:
    out_4 = scalar(
        v,
        dx,
        dt,
        source_amplitudes=source_amplitudes,
        source_locations=source_locations,
        receiver_locations=receiver_locations,
        accuracy=8,
        pml_freq=corner_freqs[1],
    )
    receiver_amplitudes_4 = out_4[-1]
    torch.save(receiver_amplitudes_4, file_path)

In [ ]:
vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.imshow(
    receiver_amplitudes_4.cpu()[0].T,
    aspect="auto",
    cmap="seismic",
    vmin=-vmax,
    vmax=vmax,
)
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(receiver_amplitudes_4.cpu()[0, 290])
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")

In [ ]:
plt.figure(figsize=(17, 6))
coherence_data_4 = receiver_amplitudes_4[0].cpu().numpy()

cov_len = 1
sig_to_noise = 0.5
noise = f.get_noise(coherence_data_4, cov_len)
noise = noise / np.max(np.abs(noise))

noise = noise * np.max(np.abs(coherence_data_4)) * sig_to_noise
coherence_data_4 = coherence_data_4 + noise

vmin, vmax = torch.quantile(
    receiver_amplitudes_4[0], torch.tensor([0.1, 0.999]).to(device)
)
plt.subplot(1, 3, 1)
plt.imshow(
    coherence_data_4.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax
)
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(noise.T, aspect="auto", cmap="seismic", vmin=-vmax, vmax=vmax)
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(coherence_data_4[19], label="Noisy Signal")
plt.plot(noise[19], label="Noise")
plt.title("Receiver 20")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend()

In [ ]:
xx = 1
win_len = 1 / xx
xx = 1
coherence_data_4 = coherence_data_4[:, : nc // xx]
noise = noise[:, : nc // xx]
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
event_detection_qr_4, _, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="qr"
)
noise_detection_qr, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="qr"
)
t1 = time.time()
qr_time = t1 - t0

t0 = time.time()
event_detection_svd_4, svds_4, frequencies = f.coherence(
    coherence_data_4, win_len, overlap, sample_interval=dt, method="svd"
)
noise_detection_svd, _, frequencies = f.coherence(
    noise, win_len, overlap, sample_interval=dt, method="svd"
)
t1 = time.time()
svd_time = t1 - t0

print("QR time: ", qr_time)
print("SVD time: ", svd_time)

In [ ]:
win_len = 1
overlap = 0
samples_per_sec = 1 / dt

t0 = time.time()
norm_win_spectra_4, frequencies = f.normalised_windowed_spectra(
    coherence_data_4, win_len, overlap, sample_interval=1 / samples_per_sec
)
t1 = time.time()
common_time = t1 - t0

welch_coherence_mat_4 = np.matmul(
    norm_win_spectra_4, np.conjugate(norm_win_spectra_4.transpose(0, 2, 1))
)
coherence_4 = np.absolute(welch_coherence_mat_4) ** 2

norm_win_spectra_noise, frequencies = f.normalised_windowed_spectra(
    noise, win_len, overlap, sample_interval=1 / samples_per_sec
)
welch_coherence_mat_noise = np.matmul(
    norm_win_spectra_noise,
    np.conjugate(norm_win_spectra_noise.transpose(0, 2, 1)),
)
coherence_noise = np.absolute(welch_coherence_mat_noise) ** 2

In [ ]:
fsize = 12
last_freq_index = -1
f_plot = frequencies
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_qr_4[1:last_freq_index],
    "-o",
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    event_detection_svd_4[1:last_freq_index],
    "-o",
    label="SVD approximation",
    color="darkviolet",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Event", fontsize=fsize)
plt.legend(fontsize=fsize)

plt.subplot(1, 2, 2)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_qr[1:last_freq_index],
    color="goldenrod",
    label="QR approximation",
)
plt.plot(
    f_plot[1:last_freq_index],
    noise_detection_svd[1:last_freq_index],
    label="SVD approximation",
    color="darkviolet",
)

plt.ylabel("Detection parameter", fontsize=fsize)
plt.xlabel("Frequency", fontsize=fsize)
plt.xticks(fontsize=fsize)
plt.yticks(fontsize=fsize)
plt.title("Noise", fontsize=fsize)
plt.legend(fontsize=fsize)

In [ ]:
i = 25

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 2)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(
    qr_signal1 / np.sum(qr_signal1), "-o", color=colors[4], label="Signal"
)
plt.plot(qr_signal2 / np.sum(qr_signal2), "-o", color=colors[0], label="Noise")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
# plt.ylim([-0.02, 0.9])
plt.legend(fontsize=tick_size)

plt.subplot(1, 3, 3)
Q, R = np.linalg.qr(norm_win_spectra_4[i])
qr_signal1 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)
Q, R = np.linalg.qr(norm_win_spectra_noise[i])
qr_signal2 = np.sum(np.multiply(R, np.conjugate(R)).real, axis=1)

plt.plot(svds_4[i] / np.sum(svds_4[i]), "-o", color=colors[4], label="Signal")
plt.ylabel("Normalized Eigenvalue", fontsize=fsize)
plt.xlabel("Snapshot in time", fontsize=fsize)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);
# plt.ylim([-0.02, 0.9])
# plt.legend(fontsize=tick_size)